In [ ]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import time
from pprint import pprint
import requests
from datetime import date, timedelta, datetime
import json
# Import API key
from config import weather_api_key

### Get Long Lat list of tuples to pass into Weather API

In [ ]:
geojson_data = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/4.5_month.geojson'

In [ ]:
response = requests.get(geojson_data).json()
features = response["features"]
test = features[0]['properties']
coords_list = []
for data in features:
    time = data['properties']['time']
    long_coords = data["geometry"]["coordinates"][0]
    lat_coords = data["geometry"]["coordinates"][1]
    coords = {
            'lat': lat_coords,
            'lng': long_coords,
            'time': time }
    
    coords_list.append(coords)
print(coords_list)
# coords_list

In [ ]:
len(coords_list)

-----

### Use Coordinates to get historical weather data.
Coordinates represent earthquake lat long location

In [ ]:
from tqdm import tqdm
from tqdm import tqdm_notebook

In [ ]:
pbar = tqdm(total=len(coords_list))

tuple_of_data = []
add_tuple = {}
counter = 0

def checkForTwo(number):
    if (int(number)) <= 1:
        number = '1'
    if (int(number)) < 10:
        number = "0" + str(number)
    else:
#         print(number)
        number = str(number)
        pass
    return number



for coordinates in coords_list:
    # Lat / long
    coords = {coordinates['lat'],coordinates['lng']}
    ######################
    ## Earthquake time conversion
    ######################
    earthquake_time = coordinates['time']
    dt3 = datetime.fromtimestamp(earthquake_time / 1000)
    # Time range end is the day of the earthquake
    time_range_end = str(dt3.year) + "-" + checkForTwo(dt3.month) + "-" + checkForTwo(dt3.day)
    
    # Time range begin is 3 days before the earthquake
    four_day_series = dt3 - timedelta(3)
    four_day_day = dt3.day
    time_range_begin = str(dt3.year) + "-" + checkForTwo(dt3.month) + "-" + checkForTwo(four_day_day)
#     print(time_range_begin)
    
    base_url = 'http://api.weatherstack.com/historical'
    params_weather = {'access_key': weather_api_key, 
                      'query': coords,
                      'historical_date_start': time_range_begin,
                        'historical_date_end': time_range_end
                 }
    response = requests.get(base_url, params=params_weather).json()
    try: 
        
        # extract results
        location = response.get('location')
#         print(location)
        # City, Country, Region
        city = location["name"]
        country = location["country"]
        region = location["region"]

        # Historical Only
        historical = response['historical']
#         print(region)
        for hist in historical:
            add_tuple = {
                'city': location.get('name'),
                'country': location["country"],
                'region': location["region"],
                'avgtemp': historical[hist]['avgtemp'],
                'date': historical[hist]['date'],
                'date_epoch': historical[hist]['date_epoch'],
                'maxtemp': historical[hist]['maxtemp'],
                'mintemp': historical[hist]['mintemp'],
                'sunhour': historical[hist]['sunhour'],
                'totalsnow': historical[hist]['totalsnow'],
                'uv_index': historical[hist]['uv_index'],
                'long': coordinates['lng'],
                'lat': coordinates['lat']
            }
            pbar.update(1)
        tuple_of_data.append(add_tuple)

    except TypeError as e:
        pbar.update(1)
#         print(e)
        continue
    pbar.update(1)
    counter += 1  
#     print(counter)
pbar.close() 
#     break
# tuple_of_data

In [ ]:
len(tuple_of_data)

## Saving to JSON

In [ ]:
from datetime import datetime
now = datetime.now()

In [ ]:
# Get time
timedate = now.strftime("%Y-%m-%d_%H_%M_%S")

# SAVE: Player_position
with open(f'../{timedate}_4_day_weather_for_earthquakes.json', 'w') as fp:
    json.dump(tuple_of_data, fp)
    

In [ ]:
# del tuple_of_data

## Opening JSON .. Do not have to run the api again.

In [ ]:
# Open raw json to parse
with open('../2020-08-12_03_17_28_4_day_weather_for_earthquakes.json', 'r') as f:
    weather_earthquake_data = json.load(f)

In [ ]:
# Dataframe to see raw(cleaned) data
all_logs = pd.DataFrame(weather_earthquake_data)
all_logs

In [ ]:
all_logs[all_logs['city'] == 'NaN']

## Upload to PostgreSQL

In [ ]:
# Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine
# Imports the methods needed to abstract classes into tables
from sqlalchemy.ext.declarative import declarative_base
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Float 
import pandas as pd

In [ ]:
# Create Weather and Earthquake Classes
# ----------------------------------
# Sets an object to utilize the default declarative base in SQL Alchemy
Base = declarative_base()

# Creates Classes which will serve as the anchor points for our Tables
class WeatherSeries(Base):
    __tablename__ = 'weatherSeries'
    id = Column(Integer, primary_key=True)
    lat = Column(String(15))
    long = Column(String(15))
#     date = Column(Integer)
#     city = Column(String(50))
#     country = Column(String(200))
#     region = Column(String(80))
#     avgtemp = Column(Integer)
#     date_epoch = Column(Integer)
#     maxtemp = Column(Integer)
#     mintemp = Column(Integer)
#     sunhour = Column(Float)
#     totalsnow = Column(Integer)
#     uv_index = Column(Integer)

# class Earthquake(Base):
#     __tablename__ = 'earthquake'
#     id = Column(Integer, primary_key=True)
#     lat = Column(Integer)
#     lon = Column(Integer)
#     date = Column(Integer)
#     mag = Column(Integer)
#     location = Column(String(255))

In [ ]:
### 
## KEEP !!
## DO NOT DELETE ALL THIS
###

# Create a Specific Instance of the "weather_data" and "earthquake_data" classes
# ----------------------------------
# Data needs to pass through this. A for loop while need to be created to consistantly send multiple files
# weather_data = WeatherSeries(
#     lat = '',
#     lon = '',
#     date = '',
#     city = '',
#     country = '',
#     region = '',
#     avgtemp = '',
#     date_epoch = '',
#     maxtemp = '',
#     mintemp = '',
#     sunhour = '',
#     totalsnow = '',
#     uv_index = '',
#     )

# earthquake_data = Earthquake(
#     lat = '',
#     lon = '',
#     date = '',
#     mag = '',
#     location = ''
#     )

In [ ]:
for k,v in all_logs.items():
#     print(k,v)
    weather_data = WeatherSeries(
            lat = log["lat"],
            long = log["long"],
#             date = 'log["date"]',
#             city = 'log["city"]',
#             country = 'log["country"]',
#             region = 'log["region"]',
#             avgtemp = 'log["avgtemp"]',
#             date_epoch = 'log["date_epoch"]',
#             maxtemp = 'log["maxtemp"]',
#             mintemp = 'log["mintemp"]',
#             sunhour = 'log["sunhour"]',
#             totalsnow = 'log["totalsnow"]',
#             uv_index = 'log["uv_index"]',
            )

    
#     test = log['city']
#     print(test)

In [ ]:
# Create Database Connection
# ----------------------------------
# Creates a connection to our DB
# Engine opens the door. Conn is the walk through sign
engine = create_engine("sqlite:///earthquake_weather.sqlite")
conn = engine.connect()

# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
# Create (if not already in existence) the tables associated with our classes.
Base.metadata.create_all(engine)

In [ ]:
# Use this to clear out the db
# ----------------------------------
# Base.metadata.drop_all(engine)

In [ ]:
# Create a Session Object to Connect to DB
# ----------------------------------
# Session is a temporary binding to our DB
# Session is the messanger-- allows to walk through the door
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [ ]:
# Add Records to the Appropriate DB
# ----------------------------------
# Use the SQL ALchemy methods to run simple "INSERT" statements using the classes and objects  
session.add(weather_data)
# session.add(earthquake_data)
session.commit()

### Test DB

In [ ]:
weather_list = session.query(WeatherSeries)
for weather in weather_list:
    print(weather.id)

In [ ]:
# Create DataFrame from sql table Weather
weather_data_df = pd.read_sql("SELECT * FROM WeatherSeries", conn)
weather_data_df

-------------

-----------

In [ ]:
# def checkEqual3(lst):
#     return lst[1:] == lst[:-1]

# def checkRowValueDups(data):
#     dictOfKeys = data.keys()
#     for column in data:
#         column_values = data[column].to_list()
#         result = checkEqual3(column_values)
# #         print('passed.')
#     if result == True:
#         all_logs_df = all_logs.iloc[0:1, 0:]
#     else:
#         print("Couldnt complete cleaning. Different value found in column where only duplicates live. Check function checkRowValueDups() for information")
#     return all_logs_df

# all_logs_df = checkRowValueDups(all_logs)

In [ ]:
# all_logs_df['2020-08-03'][0]

In [ ]:
# stuff = (list(all_logs.itertuples(index=False, name=None)))